<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda_global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 53 s (started: 2023-09-30 11:13:51 +00:00)


In [5]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! rm -f Makefile*
! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! mv pyLiBELa/src src
! mv pyLiBela/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

In [7]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [8]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()
targets = targets[:-2] #os últimos 2 são list e list.txt, só quero os pdb

In [9]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 150 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-4;
search_box = 30.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "stogo" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;
Input.grid_prefix = "McGrid"

In [10]:
#@title Docking {display-mode: "form"}

%reload_ext autotime
import os


print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));

target_cut = targets[:]

k=0
for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  mma_folder = "self_docking_mma"
  ! mkdir -p $mma_folder
  %cd $mma_folder

  subfolder = "self_docking_"+Input.energy_optimizer
 # !rm -rf McLiBELa_dock*

  if subfolder not in os.listdir("../"):
    if "McGrid.grid" in os.listdir():


      %cd ..


      !mkdir -p $subfolder
      !rm -rf $subfolder/*

      !cp self_docking_mma/McGrid.grid $subfolder/McGrid.grid
      !cp self_docking_mma/lig2.smi $subfolder/lig2.smi


      %cd $subfolder

      print()
      print()

      ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
      ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


      insmiles = open('lig2.smi', 'r');
      line=insmiles.readline().strip()
      line2 = line.split()
      ligand_smiles = line2[0]
      insmiles.close();

      lig_src =  'lig.mol2.gz'
      rec_src = 'rec.mol2.gz'

      try:

        REC = Mol2(Input, rec_src)
        RefLig = Mol2(Input, lig_src)
        Lig2 = Mol2()

        ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

      except (RuntimeError, MemoryError):
        print("Skipping, something went wrong...Check the logs.")
        log = open("log.txt", 'w')
        log.write("Something went wrong, this target was skipped.")
        log.close()
        continue


      if (ligcheck):

        Writer = WRITER(Input)
        Coord = COORD_MC()
        HB = FindHB()
        Ene  = Energy2(Input)

        for i in range(len(REC.residue_pointer)-1):
          HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
        HB.find_ligandHB(lig_src, RefLig);

        Dock = Docker(Writer)
        center = Coord.compute_com(RefLig)

        start_time = timeit.default_timer()
        Grids = Grid(Input, Writer)
        Grids.load_grids_from_file()
        time_grid = timeit.default_timer() - start_time
        print(time_grid)


        start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

        grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);


        start_time = timeit.default_timer()
        Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
        time_dock = timeit.default_timer() - start_time


        log = open("log.txt", 'a')

        header = "target\tREC.N\tRefLig.N\tLig2.N\ttime_grid\tstart_energy\tgrid_energy\ttime_dock\n"
        log.write(header)

        data = "%s\t%d\t%d\t%d\t%.3f\t%.3f\t%.3f\t%.3f\n" %(target,REC.N, RefLig.N, Lig2.N,time_grid,start_energy,grid_energy, time_dock)
        log.write(data)
        print(data)
        log.close()

  else:
    print('Target already docked!')





A saída de streaming foi truncada nas últimas 5000 linhas.
/content/drive/MyDrive/pyLiBELa/SB/1KKK
/content/drive/MyDrive/pyLiBELa/SB/1KKK/self_docking_mma
Target already docked!
310
/content/drive/MyDrive/pyLiBELa/SB/1KL6
/content/drive/MyDrive/pyLiBELa/SB/1KL6/self_docking_mma
Target already docked!
311
/content/drive/MyDrive/pyLiBELa/SB/1KQU
/content/drive/MyDrive/pyLiBELa/SB/1KQU/self_docking_mma
Target already docked!
312
/content/drive/MyDrive/pyLiBELa/SB/1KQY
/content/drive/MyDrive/pyLiBELa/SB/1KQY/self_docking_mma
Target already docked!
313
/content/drive/MyDrive/pyLiBELa/SB/1KQZ
/content/drive/MyDrive/pyLiBELa/SB/1KQZ/self_docking_mma
Target already docked!
314
/content/drive/MyDrive/pyLiBELa/SB/1KR0
/content/drive/MyDrive/pyLiBELa/SB/1KR0/self_docking_mma
Target already docked!
315
/content/drive/MyDrive/pyLiBELa/SB/1KR1
/content/drive/MyDrive/pyLiBELa/SB/1KR1/self_docking_mma
Target already docked!
316
/content/drive/MyDrive/pyLiBELa/SB/1KR6
/content/drive/MyDrive/pyLiBELa/S